In [398]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
%matplotlib inline
plt.rcParams['figure.dpi']= 150

# A simple time-dependent model of kidney

Consider a multidomain type model on a (rescaled) spatial domain $\Omega = (0,1)$, on which we have a non-dimensionalized system:
\begin{align}
    \frac{\partial \alpha_k}{\partial t}  + \mathrm{Pe}\frac{\partial}{\partial x}\left( \alpha_k u_k \right) &= - w_k,\\
    \frac{\partial\alpha_0}{\partial t}+\mathrm{Pe}\frac{\partial}{\partial x}\left( \alpha_0 u_0 \right) &=\sum_k w_k,\\
    \frac{\partial}{\partial t}\left( \alpha_k c_i^k \right)&=-\frac{\partial}{\partial x} f_i^k - g_i^k,\\
    \frac{\partial}{\partial t}\left( \alpha_0 c_i^0 \right)&=-\frac{\partial}{\partial x} f_i^0 + \sum_k g_i^k,\\
    \nu_k\left( p_k - p_0 \right) &= \frac{\alpha_k}{\bar{\alpha}_k}-1,\\
    \alpha_0 + \sum_{k} \alpha_k &= \alpha_*,
\end{align}
    where $k$ represents ascending, descending and collecting tubules of a homogeneous population of nephrons, and
\begin{align}
    \frac{\rho_\cdot}{\alpha_\cdot}u_\cdot &= -\frac{\partial p_\cdot}{\partial x},\\
    % f_i^\cdot&= -D_i^\cdot\frac{\partial c_i^\cdot}{\partial x}+\mathrm{Pe}\left( \alpha_\cdot u_\cdot c_i^\cdot \right),\\
    f_i^\cdot&= -D_i^\cdot c_i^\cdot\frac{\partial \mu_i^\cdot}{\partial x}+\mathrm{Pe}\left( \alpha_\cdot u_\cdot c_i^\cdot \right),\quad \mu_i^\cdot:=\ln c_i^k \\
    w_k&= \zeta_k\left( \psi_k-\psi_0 \right),\quad\psi_\cdot := p_\cdot - \pi_.,\quad \pi_\cdot := \frac{a_\cdot}{\alpha_\cdot}+\sum_i c_i^\cdot,\\
    g_i^k &= j_i^k+h_i^k,\quad j_i^k :=\gamma_i^k(\mu_i^k-\mu_i^0).
    % := \ln\left( \frac{c_i^k}{c_i^0} \right).
\end{align}
The subscript $i$ here represents solute species, namely, salt and urea.

We can see that the physical parameters of the dimensionless model are $\mathrm{Pe},\rho_\cdot, D_i^\cdot, \nu_k, \bar{\alpha}_k, \alpha_*,a$.
In this simulation, we will use $\alpha_* = 1$, $a=0$, and $\bar{\alpha}_k = 1/4$.
For the transport parameters, we have $\zeta_k,\gamma_i^k$.
For illustrating purpose, we assume that
\begin{equation}
    h_\mathrm{salt}^\mathrm{A} = (c_\mathrm{salt}^\mathrm{A})^+,\quad \text{on}\quad (0,1/2).
\end{equation}

In [399]:
# parameters
Pe = 1
rho = np.ones(4)
D = np.ones([2,4])
nu = np.ones(4)/20
alpha_bar = np.ones(4)*1/4
alpha_star = 1

# filtrates = np.array([1.8,0.2]) #filtrated salt and urea
filtrates = np.array([1,1]) #filtrated salt and urea

def psi(k,c,p): # c is an (2,4,N+1)-array, p is an (4,N+1)-array
    return p[k,:] - pi(k,c)

def pi(k,c): # c is an (2,4,N+1)-array
    return c[0,k,:]+c[1,k,:]

def mu(i,k,c):
    return np.log(c[i,k,:])

Let $N\in \mathbb{N}$ be the number of uniformly spaced grids in $(0,1)$, and $\delta x = 1/N$ be the spatial grid size.
Similarly, we denote $\delta t$ as the size of time steps.
We will use the notation $\alpha^{n}_{kl},c_{il}^{kn}, p_{kl}^{n}$, $k=0,\mathrm{D},\mathrm{A},\mathrm{C}$, $1\leq l\leq N$, for the discretization of $\alpha_k,c_i^k$ and $p_k$ in the $l$-th spatial grid and time $t = n\delta t$.

In [400]:
N = 20 # the number of spatial grid
dt = 0.01 # time step size
dx = 1/N # spatial grid size

zeta = np.zeros([3,N])
gamma = np.zeros([2,3,N])
for l in range(1,N):
    zeta[0,l] = 0.1
    zeta[2,l] = 0.1
    gamma[0,1,l] = 1
    if l>=round(N/2):
        gamma[1,:,l] = 1

def AHL_pump(c_salt):
    pump = np.zeros(N)
    for l in range(round(N/2)):
        pump[l] = 2*np.maximum(0,c_salt[l])
    return pump

We will use an implicit scheme to update the unknown for each time step.

First, we define difference quotient operators:
\begin{equation}
    \Delta_x^+y^n_l:= \frac{y^n_{l+1}-y^n_{l}}{\delta x},\quad \Delta_x^- y^n_l :=\frac{y^n_l-y^n_{l-1}}{\delta x},\quad \Delta_t y^n_l = \frac{y^n_l - y^{n-1}_l}{\delta t},
\end{equation}
and an average operator:
\begin{equation}
    Ay_l^n := \frac{y^n_{l+1}+y^n_l}{2}.
\end{equation}


In [401]:
def diff_x(y): 
    n = len(y)-1
    diff = np.zeros(n)
    for l in range(n):
        diff[l] = (y[l+1]-y[l])/dx
    
    return diff

# def diff_bw(y): 
#     n = len(y)-1
#     diff = np.zeros(n)
#     for l in range(1,n+1):
#         diff[l] = (y[l]-y[l-1])/dx
    
#     return diff

def diff_t(y_1,y_0): 
    return (y_1-y_0)/dt

def average(y):
    n = len(y)-1
    avg = np.zeros(n)
    for l in range(n):
        avg[l] = (y[l+1]+y[l])/2
    
    return avg

## Implicit scheme

Now, we give a method for computing residual of the implicit function for each time step.
The equations for the volumes are
\begin{gather}
    \Delta_t \alpha_{kl}^n +\mathrm{Pe}\Delta_x^-((A\alpha_{kl}^n) u_{kl}^n) = -w_{kl}^{n},\quad w_{kl}^n:=\zeta_k\left( \psi_{kl}^n-\psi_{0l}^n \right)\\
    \psi_{\cdot l}^n:= p_{.l}^n - \pi_{\cdot l}^n,\quad \pi_{\cdot l}^n:= \sum_i c_{il}^{\cdot n},\nonumber
\end{gather}
and
\begin{equation}
    \Delta_t \alpha_{0l}^n +\mathrm{Pe}\Delta_x^-(\alpha_{0l}^n u_{0l}^n) = \sum_k w_{kl}^{n}.
\end{equation}


In [402]:
def water_trans(k,c,p): 
    return zeta[k-1,:]*(psi(k,c,p) - psi(0,c,p))

def p_junction(k,alpha,p):
    if k == 1:
        n = -1
    elif k == 2:
         n = 0
    return (alpha[k,n]**2*p[k,n]/rho[k] + alpha[k+1,n]**2*p[k+1,n]/rho[k+1])/(alpha[k,n]**2/rho[k]+alpha[k+1,n]**2/rho[k+1])

def water_flow(k,alpha,p,GFR=1):
    au = np.zeros(N+1)
    au[1:-1] = -(average(alpha[k,:])**2)*diff_x(p[k,:])/rho[k]
    if k == 0:
        # au[0] = (p_vessel - p[0,0])/R_vessel
        au[0] = 0
    elif k == 1:
        au[0] = GFR
        au[-1] = 2*(alpha[k,-1]**2 /rho[k])*(p[1,-1] - p_junction(1,alpha,p))/dx
    elif k == 2:
        au[0] = 2*(alpha[k,0]**2 /rho[k])*(p_junction(2,alpha,p)-p[2,0])/dx
        au[-1] = 2*(alpha[k,-1]**2 /rho[k])*(p[2,-1] - p_junction(1,alpha,p))/dx
    elif k == 3:
        au[0] = 2*(alpha[k,0]**2 /rho[k])*(p_junction(2,alpha,p)-p[3,0])/dx
        # au[-1] = (p[3,-1] - p_papillary)/R_papillary
        au[-1] = GFR
    
    return au

def water_res(k,alpha_1,alpha_0,c,p,GFR=1):
    if k == 0:
        # sum_alpha = np.zeros(N)
        w = np.zeros(N)
        for j in range(1,4):
            w = w + water_trans(j,c,p)
            # sum_alpha = sum_alpha + alpha_1[j,:]
        # return alpha_star - sum_alpha
    else:
        w = - water_trans(k,c,p)

    alpha_t = diff_t(alpha_1[k,:],alpha_0[k,:])
    
    # alph1_avg = np.zeros(N+1)
    # alph1_avg[0] = alpha_1[k,0].copy()
    # alph1_avg[1:-1] = average(alpha_1[k,:])
    # alph1_avg[-1] = alpha_1[k,-1].copy()
    # alphu_x = diff_x(alph1_avg*water_flow(k,alpha_1,p,GFR))
    alphu_x = diff_x(water_flow(k,alpha_1,p,GFR))
    
    return alpha_t + Pe*(alphu_x) - w

Similarly for the solute equations, we have
\begin{equation}
    \Delta_t (\alpha_{kl}^n c_{il}^{kn}) = -\Delta_x^{-}f_{il}^{kn} - g_{il}^{kn},
\end{equation}

In [403]:
def solute_trans(i,k,c):
    j_ik = gamma[i,k-1,:]*(mu(i,k,c) - mu(i,0,c))
    h_ik = np.zeros(N)
    if k == 2:
        h_ik[:] = AHL_pump(c[0,2,:])

    return j_ik + h_ik

def conc_junction(i,k,c):
    if k == 1:
        n = -1
    elif k == 2:
         n = 0
    return (D[i,k]*c[i,k,n] + D[i,k+1]*c[i,k+1,n])/(D[i,k]+D[i,k+1])

def solute_flow(i,k,c,alpha,p,GFR=1):
    conc_avg = np.zeros(N+1)
    conc_avg[1:-1] = average(c[i,k,:])
    flow = water_flow(k,alpha,p,GFR)
    diffusion = np.zeros(N+1)
    diffusion[1:-1] = -D[i,k]*diff_x(c[i,k,:])
    if k == 0:
        conc_avg[0] = filtrates[i].copy()
        diffusion[0] = 2*D[i,k]*(conc_avg[0]-c[i,k,0])/dx
    elif k == 1:
        c_junc = conc_junction(i,1,c)
        conc_avg[0] = filtrates[i]
        conc_avg[-1] = c_junc.copy()
        diffusion[-1] = 2*D[i,k]*(c[i,k,-1] - c_junc)/dx
    elif k == 2:
        c_junc_1 = conc_junction(i,1,c)
        c_junc_0 = conc_junction(i,2,c)
        conc_avg[0] = c_junc_0.copy()
        conc_avg[-1] = c_junc_1.copy()
        diffusion[0] = 2*D[i,k]*(c_junc_0 - c[i,k,0])/dx
        diffusion[-1] = 2*D[i,k]*(c[i,k,-1] - c_junc_1)/dx
    elif k == 3:
        c_junc = conc_junction(i,2,c)
        conc_avg[0] = c_junc.copy()
        conc_avg[-1] = c[i,3,-1]
        diffusion[0] = 2*D[i,k]*(c_junc - c[i,k,0])/dx

    advection = Pe*(flow*conc_avg)

    return diffusion + advection

def solute_res(i,k,alpha_1,alpha_0,c_1,c_0,p,GFR=1):
    alphc_t = diff_t(alpha_1[k,:]*c_1[i,k,:],alpha_0[k,:]*c_0[i,k,:])
    f_x = diff_x(solute_flow(i,k,c_1,alpha_1,p,GFR))
    g = np.zeros(N)
    if k == 0:
        for j in range(1,4):
            g = g + solute_trans(i,j,c_1)
    else:
        g = -solute_trans(i,k,c_1)

    return alphc_t + f_x - g

For the pressure-compliance relations,

In [404]:
def pressure_res(k,alpha,p):
    if k == 0:
        res = alpha[0,:]/alpha_star - 1
        for j in range(1,4):
            res = res + alpha_bar[j]*(1+nu[j]*(p[j,:]- p[0,:]))/alpha_star
        return res
    else:
        return p[k,:]-p[0,:]-(alpha[k,:]/alpha_bar[k] -1)/nu[k]

Finally, we put everything together:

In [405]:
def implicit_func_res(y_1,y_0,GFR=1): # y is a 16*N-array
    alpha_1 = y_1[:4*N].reshape([4,N])
    alpha_0 = y_0[:4*N].reshape([4,N])
    c_1 = y_1[4*N:(4+(2*4))*N].reshape([2,4,N])
    c_0 = y_0[4*N:(4+(2*4))*N].reshape([2,4,N])
    p = y_1[12*N:(12+4)*N].reshape([4,N])
    
    res = np.zeros(16*N)
    res_alpha = res[:4*N].reshape([4,N])
    res_c = res[4*N:(4+(2*4))*N].reshape([2,4,N])
    res_p = res[12*N:(12+4)*N].reshape([4,N])

    for k in range(4):
        res_alpha[k,:] = water_res(k,alpha_1,alpha_0,c_1,p,GFR)
        for i in range(2):
            res_c[i,k,:] = solute_res(i,k,alpha_1,alpha_0,c_1,c_0,p,GFR)
        res_p[k,:] = pressure_res(k,alpha_1,p)

    return res

## Simulation

We gradually increase GFR from 0 to 0.02 and then hold at 0.02.

## Initial condition

We have to ensure that the pressure-compliance relation is satisfied.
<!-- Then, given the initial concentrations of salt and urea, we compute the solute and water flow, $\alpha_ku_k,f_i^k$. -->

In [406]:
y_init = np.zeros(16*N)
alpha_init = y_init[:4*N].reshape([4,N])
c_init = y_init[4*N:(4+(2*4))*N].reshape([2,4,N])
p_init = y_init[12*N:(12+4)*N].reshape([4,N])

for i in range(2):
    c_init[i,:,:] = filtrates[i]

alpha_init[:,:] = 1/4
p_init[:,:] = 10

def solve_pressure(y,alpha):
    p = y.copy().reshape(4,N)
    res = np.zeros(4*N)
    for k in range(4):
        res[k*N:(k+1)*N] = pressure_res(k,alpha,p)
    return res

y_init[12*N:(12+4)*N] = fsolve(solve_pressure, y_init[12*N:(12+4)*N], args=alpha_init)

p_papillary = p_init[3,-1].copy()
# p_papillary = 0
p_vessel = p_init[0,0].copy()
R_papillary = 10
R_vessel = 10

In [416]:
y = y_init.copy()

steps = 20
dt = 0.1

for n in range(steps):
    GFR = .5*np.tanh(n/10)
    # GFR = np.tanh(n/10)*(np.sin(n*np.pi/20))/4
    y = np.vstack((y,np.zeros(16*N)))
    y_0 = y[-2,:]
    y_1 = y[-1,:]
    y_1[:] = y_0[:].copy()
    y_1[:] = fsolve(implicit_func_res,y_1,args=(y_0,GFR))

y_alpha = y[:,:4*N].reshape([steps+1,4,N])
y_c = y[:,4*N:(4+(2*4))*N].reshape([steps+1,2,4,N])
y_p = y[:,12*N:(12+4)*N].reshape([steps+1,4,N])

Let's plot the solution.